# Summarizing texts


In [6]:
from dotenv import load_dotenv
import os

load_dotenv("secrets.env")

os.getenv("OPENAI_API_KEY") is not None

True

In [3]:
from pathlib import Path

docs_dir = Path(
    "/Users/f006pfk/Dropbox (Dartmouth College)/RR-workshop-data/federalist-papers/split"
)

## Loading text documents


In [5]:
with open(docs_dir / "federalist_1.txt") as f:
    doc = f.read()

print(doc[:200])

 1

General Introduction

For the Independent Journal. Saturday, October 27, 1787


HAMILTON

To the People of the State of New York:

AFTER an unequivocal experience of the inefficacy of the subsisti


In [17]:
from langchain.document_loaders import DirectoryLoader

doc_loader = DirectoryLoader(docs_dir, show_progress=True)
docs = doc_loader.load()

100%|██████████| 86/86 [00:08<00:00, 10.47it/s]


## Stuffing


In [22]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-16k")

In [23]:
for doc in docs[:3]:
    print("-" * 30)
    print(
        llm.predict("Summarize the following text in 200 words: \n" + doc.page_content)
    )

------------------------------
In this text, James Madison discusses the powers granted to the government by the Constitution in two general points of view. The first point relates to the amount of power vested in the government and the restraints placed on the states. Madison questions whether any of the transferred powers are unnecessary or improper, and whether the total power is dangerous to the states. He argues that those who criticize the extensive powers of the government have not considered whether these powers are necessary means to achieve a necessary end. He also acknowledges that every power has the potential for abuse, but that this should not overshadow the public good that can come from the exercise of that power. 

Madison then discusses the powers granted to the government, divided into different classes. These include security against foreign danger, regulation of foreign intercourse, maintenance of harmony among the states, general utility, restraint of states from 

### Using a chain


In [27]:
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain

In [40]:
# Define prompt
prompt_template = "Write a concise summary of the following:{text}"

prompt = PromptTemplate.from_template(prompt_template)

# Define LLM chain
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k")
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")

print(stuff_chain.run(docs[:1]))

This article discusses the number of representatives in the House of Representatives and addresses concerns about the smallness of the number. The author argues that the small number of representatives is not a danger to public liberty and that the people of America would not choose representatives who would betray their trust. The article also dismisses concerns about corruption from foreign gold or other branches of the federal government. The author concludes that the number of representatives proposed by the Constitution is sufficient to protect the liberties of America.


### Convenience wrapper


In [39]:
from langchain.chains.summarize import load_summarize_chain

chain = load_summarize_chain(llm=llm, chain_type="stuff")

chain.run(docs[:1])

'This article, written by James Madison in 1788, discusses the number of representatives in the House of Representatives and addresses concerns about the smallness of the number. Madison argues that the small number of representatives is not a threat to public liberty and that the people of America, as well as the state legislatures, would prevent any abuse of power. He also dismisses concerns about corruption from foreign influence or other branches of government. Madison concludes that the liberties of America are not at risk with the number of representatives proposed by the Constitution.'

## Map-reduce


In [43]:
from langchain.chains import ReduceDocumentsChain, MapReduceDocumentsChain

In [51]:
# Map
map_template = "Write a concise summary of the following: {docs}"
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

In [52]:
# Reduce
reduce_template = "Distill the following summaries into a final, consolidated summary of the main topics: {doc_summaries}"
reduce_prompt = PromptTemplate.from_template(reduce_template)
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

In [53]:
# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="doc_summaries"
)

In [54]:
# Combines and iteravely reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=4000,
)

In [55]:
# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)

In [57]:
print(map_reduce_chain.run(docs[:10]))

This consolidated summary covers various topics discussed in the given articles. It addresses concerns about the number of representatives in the House of Representatives, arguing that the small number is not a danger to public liberty. The importance and necessity of the powers conferred by the Constitution are defended, including the power to declare war, raise armies, and regulate interstate relations. The executive authority of the President is compared to that of a king, concluding that claims of excessive power are unfounded. The judiciary department and the role of State courts are discussed, emphasizing the need for collaboration with federal courts. The lack of a constitutional provision for trial by jury in civil cases is addressed, arguing that it is not essential to liberty. The mode of electing the President is examined, with the current system being praised for its representation of the people and minimizing corruption. The authority of the convention to propose a mixed C